In [ ]:
#| default_exp argument_validators

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from typing import Protocol, Any, runtime_checkable
from collections.abc import Callable
from pymoq.core import AnyCallable

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import test_fail

# Validators

> Collection of methods to validate specific call arguments.

Goal: Evaluate whether a call like `f(1,"s")` matches any signature-pattern. A signature pattern might be defined like `f(1, str)`. This should match any call that passes the exact value one for the first argument and any object of type str in the second.

## Argument validators

We break the task down to validating a single argument. The signature of such an ArgumentValidator should look like :

In [ ]:
#| export
@runtime_checkable
class ArgumentValidator(Protocol):
    "Interface for all argument validators."
    
    @property
    def name() -> str:
        "Name of the argument in the signature"
        
    @property
    def position() -> int:
        "Position of the argument in the signature"
        
    def is_valid(self, argument: Any) -> bool:
        ...

The most flexibility can be achieved by constructing an ArgumentValidator that evaluates an arbitrary function:

In [ ]:
#| export
class ArgumentFunctionValidator:
    "Validate an argument by evaluating an arbitrary function"
    def __init__(self, func: AnyCallable[bool], name: str, position: int, display:str|None = None):
        self._func = func
        self._name = name
        self._position = position
        self._display = display
        
    @property
    def name(self) -> str:
        return self._name
    
    @property
    def position(self) -> int:
        return self._position
        
    def is_valid(self, argument: Any) -> bool:
        return self._func(argument)
    
    def __str__(self):
        if self._display is None:
            return f'ArgumentFunctionValidator(name:{self.name}, position={self.position})'
        return f'ArgumentFunctionValidator(argument_name:{self.name}, position={self.position}): {self._display}'
    def __repr__(self): return str(self)
    
assert isinstance(ArgumentFunctionValidator, ArgumentValidator), "ArgumentFunctionValidator does not implement the ArgumentValidator-Protocol"

This could now be used like:

In [ ]:
any_int = ArgumentFunctionValidator(lambda v: isinstance(v, int), "firstArgument", 0, display='AnyInt()')
any_int

ArgumentFunctionValidator(argument_name:firstArgument, position=0): AnyInt()

In [ ]:
assert any_int.is_valid(1)
assert not any_int.is_valid(1.1)
assert not any_int.is_valid("string")
assert str(any_int)=='ArgumentFunctionValidator(argument_name:firstArgument, position=0): AnyInt()'

In later stages there should be convenience methods around creating such argument validators. E.g. `from_type(some_type)` for making the above easier.

### Ease of use: Construction from arguments

In [ ]:
#| export
def argument_validator_from_argument(argument: Any, name:str, position: int, verbose:bool=False) -> ArgumentValidator:
    if verbose: print(f"Constructing ArgumentValidatorFrom {argument}")
    match argument:
        case ArgumentValidator():
            return argument
        case type():
            return ArgumentFunctionValidator(lambda v: isinstance(v, argument), name=name, position=position, display=f'any_{argument.__name__}')
        case Callable():
            if hasattr(argument, 'display'):
                display = argument.display
            else:
                display = 'callable()'
            
            return ArgumentFunctionValidator(argument, name=name, position=position, display=display)
    
    return ArgumentFunctionValidator(lambda v: v==argument, name=name, position=position, display=f'== {argument}')

##### ArgumentValidator

Passing a valid `ArgumentValidator` simply returns it:

In [ ]:
assert argument_validator_from_argument(any_int, any_int.name, 0) == any_int

##### Callable

Passing a callable constructs an `ArgumentFunctionValidator`:

In [ ]:
arg_val = argument_validator_from_argument(lambda v: isinstance(v, int), 'a', 0)
arg_val

ArgumentFunctionValidator(argument_name:a, position=0): callable()

In [ ]:
check_type = lambda v: isinstance(v, int)
check_type.display = 'any_int'

arg_val = argument_validator_from_argument(check_type, 'a', 0)
arg_val

ArgumentFunctionValidator(argument_name:a, position=0): any_int

In [ ]:
assert isinstance(arg_val, ArgumentValidator)
assert arg_val.is_valid(1)
assert not arg_val.is_valid("1")

##### Type

In [ ]:
arg_val = argument_validator_from_argument(int, name='a', position=0)
arg_val

ArgumentFunctionValidator(argument_name:a, position=0): any_int

In [ ]:
assert isinstance(arg_val, ArgumentValidator)
assert arg_val.is_valid(1)
assert not arg_val.is_valid("1")

##### Non-callable, non-type

Passing a non-callable assumes that the value should be compared against, i.e. it's a constant:

In [ ]:
arg_val = argument_validator_from_argument(123, name='a', position=0)
arg_val

ArgumentFunctionValidator(argument_name:a, position=0): == 123

In [ ]:
assert isinstance(arg_val, ArgumentValidator)
assert arg_val.is_valid(123)
assert not arg_val.is_valid(124)
assert str(arg_val)=='ArgumentFunctionValidator(argument_name:a, position=0): == 123'

## Special Validators

In [ ]:
#| export
AnyArg = lambda: lambda v: True
AnyArg.display = 'any()'

In [ ]:
#| export
class AnyInt:
    "Special validator that provides methods for integers"
    def __init__(self,name: str, position: int, display:str|None = None):
        self._name = name
        self._position = position
        self._display = display
        
        self._validators: list[ArgumentValidator] = [argument_validator_from_argument(int, name=name, position=position)]
        self._validator_names: list[str] = ['AnyInt()']
    
    @property
    def name(self) -> str:
        return self._name
    
    @property
    def position(self) -> int:
        return self._position
    
    def greather_than(self, lower: int) -> "AnyInt":
        greather_func = lambda value: lower<value
        self._validator_names.append(f"greather_than({lower})")
        
        self._validators.append(argument_validator_from_argument(greather_func, name=self._name, position=self._position))
        return self
    
    def greather_than_or_equal(self, lower: int) -> "AnyInt":
        greather_func = lambda value: lower<=value
        self._validator_names.append(f"greather_than_or_equal({lower})")
        
        self._validators.append(argument_validator_from_argument(greather_func, name=self._name, position=self._position))
        return self
    
    def less_than(self, upper: int) -> "AnyInt":
        less_func = lambda value: value<upper
        self._validator_names.append(f"less_than({upper})")
        
        self._validators.append(argument_validator_from_argument(less_func, name=self._name, position=self._position))
        return self
    
    def less_than_or_equal(self, upper: int) -> "AnyInt":
        less_func = lambda value: value<=upper
        self._validator_names.append(f"less_than_or_equal({upper})")
        
        self._validators.append(argument_validator_from_argument(less_func, name=self._name, position=self._position))
        return self
        
    def is_valid(self, argument: Any) -> bool:
        return all(validator.is_valid(argument) for validator in self._validators)
    
    def __str__(self):
        return '.'.join(self._validator_names)
    
    def __repr__(self): return str(self)

In [ ]:
a = AnyInt('a', 0)

assert a.is_valid(1)
assert not a.is_valid("1")
a

AnyInt()

In [ ]:
b = AnyInt('b', 1).greather_than(5)

assert b.is_valid(6)
assert not b.is_valid(5)
b

AnyInt().greather_than(5)

In [ ]:
b = AnyInt('b', 1).greather_than_or_equal(5)

assert b.is_valid(5)
assert not b.is_valid(4)
b

AnyInt().greather_than_or_equal(5)

In [ ]:
c = AnyInt('c', 2).less_than(5)

assert c.is_valid(4)
assert not c.is_valid(5)
c

AnyInt().less_than(5)

In [ ]:
c = AnyInt('c', 2).less_than_or_equal(5)

assert c.is_valid(5)
assert not c.is_valid(6)
c

AnyInt().less_than_or_equal(5)

# Build library

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()